In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
from scipy.special import logsumexp
import pymc3 as pm
import theano.tensor as TT

In [2]:
def generate_examples(p_1, n_obj, cfer_p_00, cfer_p_11):
    #for now, assume everyone classifies everything
    np.random.seed(0)
    true_classes = np.random.choice(2, p=[1-p_1, p_1], size=n_obj)
    
    n_cfer = len(cfer_p_00)
    
    p_obs_1 = np.where(true_classes[:, None]==1, cfer_p_11[None, :], (1-cfer_p_00[None, :]))
    obs_classes = np.where(np.random.uniform(0, 1, [n_obj, n_cfer]) < p_obs_1, 1, 0)
    
    return true_classes, obs_classes.T

In [3]:
p_1_true = 0.2
p_1_train = 0.5

cfer_probs = np.asarray([[0.9, 0.7],
                         [0.7, 0.9],
                         [0.8, 0.8],
                         [0.3, 0.9],
                         [0.4, 0.7],
                         [0.9, 0.9]])

n_real = 10000
true_fs = np.zeros(n_real)
obs_fs = np.zeros(n_real)

n_obj = 200
true, obs = generate_examples(p_1_true, n_obj, cfer_probs[:, 0], cfer_probs[:, 1])

n_train = 20
true_train, obs_train = generate_examples(p_1_train, n_train, cfer_probs[:, 0], cfer_probs[:, 1])

#for real_i in range(n_real):
#    true, obs = generate_examples(p_0, n_obj, cfer_probs[:, 0], cfer_probs[:, 1])
#    true_fs[real_i] = 1-np.mean(np.random.choice(true, size=n_obj, replace=True))

In [4]:
def make_flat(observations):
    n_cfers, n_obj = observations.shape
    obs_flat = observations.ravel()
    
    cfer_ids = np.zeros([n_cfers, n_obj], dtype='int')
    for i in range(n_cfers):
        cfer_ids[i] = i
    cfer_ids = cfer_ids.ravel()
    
    obj_ids = np.zeros([n_cfers, n_obj], dtype='int')
    for j in range(n_obj):
        obj_ids[:, j] = j
    obj_ids = obj_ids.ravel()
    
    return obs_flat, obj_ids, cfer_ids

In [5]:
def get_beta_pars(truths, observed):
    t0 = (truths==0)
    t1 = (truths==1)
    o0 = (observed==0)
    o1 = (observed==1)
    
    a_00 = np.count_nonzero(t0 & o0, axis=1) + 1
    b_00 = np.count_nonzero(t0 & o1, axis=1) + 1
    b_11 = np.count_nonzero(t1 & o0, axis=1) + 1
    a_11 = np.count_nonzero(t1 & o1, axis=1) + 1
    
    return a_00, b_00, a_11, b_11


In [6]:
#bayes-net style, without marginalization over outcomes
p_1f0 = 1-cfer_probs[:, 0] #true 0 -> obs 1
p_1f1 = cfer_probs[:, 1] #true 1 -> obs 1
n_cfers = len(p_1f0)


### go flat; for now, everyone classifies everything

##the "real" observations
obs_classes, obj_ids, cfer_ids = make_flat(obs)

##the training set
a_00, b_00, a_11, b_11 = get_beta_pars(true_train, obs_train) 


with pm.Model() as explicit_model:
    p_1 = pm.Uniform('p1', 0, 1)
    p_00_var = pm.Beta('p00', alpha=a_00, beta=b_00, shape=n_cfers)
    p_11_var = pm.Beta('p11', alpha=a_11, beta=b_11, shape=n_cfers)
    
    #the "real" data
    true_class = pm.Bernoulli('true_class', p_1, shape=n_obj)
    N_1_true = pm.Deterministic('N1true', TT.sum(true_class))
    p_obs_1 = TT.switch(TT.eq(true_class[obj_ids], 1), p_11_var[cfer_ids], 1-p_00_var[cfer_ids])
    observed = pm.Bernoulli('observed', p_obs_1, observed=obs_classes)
    

In [ ]:
with explicit_model:
    trace = pm.sample(draws=5000)
    
# Needed to add in a function that will output merger fraction, error on merger fraction, probability per galaxy, error on probability per galaxy.
# Make the call similar to the get_merger_probability call...

# turn this into get_merger_prob2(Mhats, r_Ms, r_Is) to output the merger fraction, error on merger fraction, prob per galaxy, err on prob per gal.

def get_merger_prob(Mhats, r_Ms, r_Is):
    '''
    Given people's mesurements, Mhat, and their accuracy at making measurements,
    r_M and r_I, we can get the probability that a given galaxy is a merger.
    '''

    P_M = 1
    P_I = 1
    for i in range(len(Mhats)):
        if Mhats[i] == 0:
            # the surveyor measured an isolated galaxy
            # probability that we measured isolated galaxy given that it's a
            # merger
            P_Ihat_M = 1-r_Ms[i]
            # probability that we measured isolated galaxy given that it's
            # isolated
            P_Ihat_I = r_Is[i]
            P_M *= P_Ihat_M
            P_I *= P_Ihat_I
        else:
            # the surveyor measured a merger
            # probability that we measured merger galaxy given that it's a
            # merger
            P_Mhat_M = r_Ms[i]
            # probability that we measured a merger given that it's
            # isolated
            P_Mhat_I = 1-r_Is[i]
            P_M *= P_Mhat_M
            P_I *= P_Mhat_I
    return P_M/(P_M+P_I)



Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [p11, p00, p1]
>BinaryGibbsMetropolis: [true_class]
Sampling 4 chains, 0 divergences: 100%|██████████| 22000/22000 [02:09<00:00, 170.30draws/s]


In [ ]:
with explicit_model:
    pm.traceplot(trace, var_names=['p1', 'p00', 'p11'])

In [ ]:
true.sum(), p_1_true

In [ ]:
cfer_probs[:, 1], cfer_probs[:, 0],

In [ ]:
obs.shape

In [ ]:
tc_samples = trace.get_values('true_class')
plt.hist(tc_samples.mean(axis=0))

In [ ]:
(tc_samples.mean(axis=0) > 0.5).mean() 